In [2]:
# Global imports and settings

# Matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)

# Print options
import numpy as np
np.set_printoptions(precision=3)

from notebook.services.config import ConfigManager
cm = ConfigManager()
_ = cm.update('livereveal', {'width': 1440, 'height': 720 })

<center>
    <img src="img/scikit-learn-logo.png" width="40%" />
    <br />
    <h1>An introduction to Machine Learning with Scikit-Learn</h1>
    <br /><br />
    Gilles Louppe (<a href="https://twitter.com/glouppe">@glouppe</a>), November 12, 2015
    <br /><br />
    Data Science @ LHC 2015, CERN
</center>

## Prerequisites 

In [3]:
# This is an IPython notebook, with executable Python code inside
from __future__ import braces

SyntaxError: not a chance (<ipython-input-3-8ea16afe3b06>, line 2)

- Materials at  <a href="https://github.com/glouppe/tutorial-sklearn-dslhc2015">https://github.com/glouppe/tutorial-sklearn-dslhc2015</a>
    
- Require a Python distribution with scientific packages (NumPy, SciPy, Scikit-Learn, Pandas)

- ... or Anaconda http://continuum.io/downloads

    

# Outline

* Scikit-Learn and the scientific ecosystem in Python
* Classification
* Model evaluation and selection
* Regression
* Outlier detection
* Summary

# Scikit-Learn

## Overview

* Machine learning library written in __Python__
* __Simple and efficient__, for both experts and non-experts
* Classical, __well-established machine learning algorithms__
* Shipped with <a href="http://scikit-learn.org/dev/documentation.html">documentation</a> and <a href="http://scikit-learn.org/dev/auto_examples/index.html">examples</a>
* __BSD 3 license__

## Community driven development

- 10-15~ core developers (mostly researchers)
- 250+ occasional contributors
- __All working together__ on [GitHub](https://github.com/scikit-learn/scikit-learn)
- Emphasis on __keeping the project maintainable__
    - Style consistency
    - Unit-test coverage
    - Documentation and examples
    - Code review
- Join us!

## Python stack for data analysis

- The __open source__ Python ecosystem provides __a standalone, versatile and powerful scientific working environment__, including: [NumPy](http://numpy.org), [SciPy](http://scipy.org), [IPython](http://ipython.org), [Matplotlib](http://matplotlib.org), [Pandas](http://pandas.pydata.org/), _and many others..._

<center> 
<img src="img/scikit-learn-logo.png" style="max-width: 120px; display: inline" />
<img src="img/numpy-logo.png" style="max-width: 120px; display: inline" />
<img src="img/scipy-logo.png" style="max-width: 120px; display: inline" />
<img src="img/ipython-logo.jpg" style="max-width: 120px; display: inline" />
<img src="img/matplotlib-logo.png" style="max-width: 120px; display: inline"/>
<img src="img/pandas-logo.png" style="max-width: 120px; display: inline" />
</center>

- Scikit-Learn builds upon NumPy and SciPy and __complements__ this scientific environment with machine learning algorithms;
- By design, Scikit-Learn is __non-intrusive__, easy to use and easy to combine with other libraries;
- Core algorithms are implemented in low-level languages.

## Algorithms

__Supervised learning:__

* Linear models (Ridge, Lasso, Elastic Net, ...)
* Support Vector Machines
* Tree-based methods (Random Forests, Bagging, GBRT, ...)
* Nearest neighbors
* Neural networks 
* Gaussian Processes
* Kernel methods

<center><a href="http://scikit-learn.org/dev/auto_examples/classification/plot_classifier_comparison.html"><img src="img/classifiers.png" width="90%" /></a>
<em>A comparison of (some of the) classifiers in Scikit-Learn</em></center><br />

__Unsupervised learning:__

* Clustering (KMeans, Ward, ...)
* Matrix decomposition (PCA, ICA, ...)
* Density estimation
* Outlier detection

__Model selection and evaluation:__

* Cross-validation
* Grid-search
* Lots of metrics

_... and many more!_ (See our [Reference](http://scikit-learn.org/stable/modules/classes.html))

# Classification

## Input data

Data comes as a finite learning set ${\cal L} = (X, y)$ where
* Input samples are given as an array $X$ of shape `n_samples` $\times$ `n_features` <br />
  _E.g. samples are events and features are their kinematic properties._

* Output values are given as an array $y$ <br />
  _E.g. whether the event is background or signal._

In [2]:
X = np.random.rand(4, 3)
y = np.random.randint(0, 2, len(X))

In [ ]:
# E.g. for n_samples = 4, n_features = 3
print("X =")
print(X)
print("y =", y)

## Loading data with `pandas`

`pandas` is a library providing easy-to-use data structures and data analysis tools. 

In [ ]:
# Download simulated events from https://archive.ics.uci.edu/ml/datasets/SUSY

# Load CSV file with pandas
import pandas as pd 
df = pd.read_csv("SUSY.csv", header=None)     
df.columns = [# 0 = background, 1 = signal
              "target", 
              # Kinematic properties
              "lepton 1 pT", "lepton 1 eta", "lepton 1 phi", 
              "lepton 2 pT", "lepton 2 eta", "lepton 2 phi", 
              "missing energy magnitude", "missing energy phi", 
              # Derived features
              "MET_rel","axial MET", "M_R", "M_TR_2", "R", "MT2", 
              "S_R", "M_Delta_R", "dPhi_r_b", "cos(theta_r1)"]
df.target = df.target.astype(int)

## Exploration and visualization

In [ ]:
df.head()  # df.describe(), df.info(), ...

In [ ]:
df_sample = df[:10000]     # Draw a sample to make things faster

# Visualize samples through a scatter matrix
colors = ["blue", "red"]   # blue = background, red = signal
features = ["lepton 1 pT", "lepton 1 eta", "missing energy magnitude", "R"]
_ = pd.scatter_matrix(df_sample[features], marker="x", 
                      c=df_sample.target.apply(lambda x: colors[x]))

Need more? See `pandas` <a href="http://pandas.pydata.org/pandas-docs/stable/visualization.html">visualization</a> documentation.

## Data in Scikit-Learn

- Input data = Numpy arrays or sparse matrices ;
- Leverage efficient vector operations ;
- Keep code short and readable. 

In [ ]:
# Get data as Numpy arrays from the pandas data frame
X = df_sample.drop("target", axis=1).values
y = df_sample.target.values

print(X[:5])
print(X.shape)
print(X.dtype)

## Supervised learning 

The goal of supervised learning is to build an estimator $\varphi_{\cal L}: {\cal X} \mapsto {\cal Y}$ minimizing

$$
Err(\varphi_{\cal L}) = \mathbb{E}_{X,Y}\{ L(Y, \varphi_{\cal L}(X)) \}.
$$

where $L$ is a loss function (e.g., the zero-one loss for classification).

## Decision trees

A decision tree is a cut-based partition of the input space, where each region (= leaf) is fit with a (simple) predictive model.

<center>
    <img src="img/tree-partition.png" width="39%" style="display:inline" />
    <img src="img/tree-simple.png" width="60%" style="display:inline" />
</center>
<small>
<pre>
def build(L):
    create node t
    if the stopping criterion is True:
        assign a predictive model to t
    else:
        Find the best binary split L = L_left + L_right
        t.left = build(L_left)
        t.right = build(L_right)
    return t     
</pre>
</small>

## A simple and unified API

All learning algorithms in scikit-learn, including decision trees, share a uniform and limited API consisting of complementary interfaces:

- an `estimator` interface for building and fitting models;
- a `predictor` interface for making predictions;
- (a `transformer` interface for converting data.)

### Estimators

An estimator is any object that learns from data; it may be a classification, regression or clustering algorithm or a transformer that extracts/filters useful features from raw data.

In [ ]:
class Estimator(object):
    def fit(self, X, y=None):
        """Fits estimator to data."""
        # set state of ``self``
        return self
            
    def predict(self, X):
        """Predict response of ``X``."""
        # compute predictions ``pred``
        return pred

In [ ]:
# Import the decision tree class
from sklearn.tree import DecisionTreeClassifier  # Change this to try 
                                                 # something else

# Set hyper-parameters, for controlling the learning algorithm
clf = DecisionTreeClassifier(criterion="entropy")

# Learn a model from training data
clf.fit(X, y)

In [ ]:
# Estimator state is stored in instance attributes
clf.tree_

### Predictors

In [ ]:
# Make predictions 
print(clf.predict(X[:5])) 

In [ ]:
# Compute class probabilities
print(clf.predict_proba(X[:5]))

## Visualization

In [ ]:
from sklearn.tree import export_graphviz
export_graphviz(clf, out_file="tree.dot", 
                feature_names=df.columns[1:], max_depth=2)
!dot -Tpng -o tree.png tree.dot
from IPython.display import Image
Image("tree.png")

## Pros and cons of decision trees

- _Non-parametric_ model, proved to be consistent.
- Support _heterogeneous data_ (continuous, ordered or categorical variables).
- Flexibility in loss functions (but choice is limited).
- Fast to train, fast to predict.
- Easily _interpretable_.
- Low bias, but usually high variance

# Model evaluation and selection

## Evaluation

In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
clf = DecisionTreeClassifier(criterion="entropy", 
                             random_state=1).fit(X_train, y_train)
print("Training accuracy =", clf.score(X_train, y_train))   # Biased estimate
print("Test accuracy =", clf.score(X_test, y_test))         # Unbiased estimate


Beware of bias when you estimate model performance:
- Training score is often an optimistic estimate of the true performance;
- The same data should not be used both for training and evaluation.

When ${\cal L}$ is small, prefer K-Fold cross-validation instead of train-test split for more accurate estimates.

<center><img src="img/kfold.jpg" width="70%" /></center>

In [ ]:
from sklearn.cross_validation import KFold

for train, test in KFold(n=len(X), n_folds=5, random_state=42):
    print(train)
    print(test)
    print()

In [ ]:
scores = []

for train, test in KFold(n=len(X), n_folds=5, random_state=42):
    X_train, y_train = X[train], y[train]
    X_test, y_test = X[test], y[test]
    clf = DecisionTreeClassifier(criterion="entropy", 
                                 random_state=1).fit(X_train, y_train)
    scores.append(clf.score(X_test, y_test))

print("%f +-%f" % (np.mean(scores), np.std(scores)))

In [ ]:
# Shortcut
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(DecisionTreeClassifier(criterion="entropy", 
                                                random_state=1), 
                         X, y, cv=KFold(n=len(X), n_folds=5, random_state=42), 
                         scoring="accuracy")
print("%f +-%f" % (np.mean(scores), np.std(scores)))

## Under- and over-fitting

- Under-fitting: the model is too simple and does not capture the true relation between X and Y.
- Over-fitting: the model is too specific to the training set and does not generalize.

In [ ]:
from sklearn.learning_curve import validation_curve

param_range = range(1, 16)
train_scores, test_scores = validation_curve(
    DecisionTreeClassifier(), X, y, 
    param_name="max_depth", 
    param_range=param_range, cv=5, n_jobs=-1)

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.xlabel("max_depth")
plt.ylabel("score")
plt.xlim(min(param_range), max(param_range))
plt.plot(param_range, train_scores_mean, color="red", label="training score")
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2, color="red")
plt.plot(param_range, test_scores_mean, color="blue", label="test score")
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2, color="blue")
plt.legend(loc="best")

In [ ]:
# Best trade-off
print("max_depth = %d, accuracy = %f" % (param_range[np.argmax(test_scores_mean)],
                                         np.max(test_scores_mean)))

## Hyper-parameter search

- Learning algorithms are not black boxes. 
- Finding good hyper-parameters is crucial to control under- and over-fitting, hence achieving better performance.
- This is automatized with the `GridSearchCV` estimator.

In [ ]:
from sklearn.grid_search import GridSearchCV
grid = GridSearchCV(DecisionTreeClassifier(),
                    param_grid={"max_depth": range(1, 16),
                                "criterion": ["gini", "entropy"],
                                "min_samples_leaf": [1, 5, 10, 50]},
                    scoring="accuracy",
                    cv=5, n_jobs=-1)
grid.fit(X, y)

# Warning: don't report these numbers in your experiments!
print("Best score = %f, Best parameters = %s" % (grid.best_score_, 
                                                 grid.best_params_))

## Selection and evaluation, _simultaneously_

- The resulting `grid.best_estimator_` model is not independent from `grid.best_score_` since its construction was guided by the maximization of this quantity. 

- As a result, the optimized `grid.best_score_` accuracy _may_ in fact be a biased, optimistic, estimate of the true performance of the model. 

In [ ]:
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import GridSearchCV

scores = cross_val_score(
            GridSearchCV(DecisionTreeClassifier(),
                         param_grid={"max_depth": range(1, 16),
                                     "criterion": ["gini", "entropy"],
                                     "min_samples_leaf": [1, 5, 10, 50]},
                         scoring="accuracy",
                         cv=5, n_jobs=-1), 
            X, y, cv=5, scoring="accuracy")

# Unbiased estimate of the accuracy
print("%f +-%f" % (np.mean(scores), np.std(scores)))

# Regression

# Outlier detection

# Ensemble methods

## Random Forests

- Single decision trees suffer from high variance. 
- This can be fixed by combining several randomized trees into a single model. 

<center>
    <img src="img/forest.png" />
</center>

- Randomization strategies:
    * Bootstrap samples
    * Random selection of $K$ split features
    * Random selection of cut-off threshold
- From the bias-variance decomposition, it can be shown that the resulting generalization performance is better.



In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000).fit(X_train, y_train)
print("Training accuracy =", rf.score(X_train, y_train)) 
print("Test accuracy =", rf.score(X_test, y_test))   

## Variable importances

- Scores derived from a tree-based model to assess the individual importance of input features.
- Can be accessed through  `estimator.feature_importances_`.

In [ ]:
# Feature importances from totally randomized trees (TRT)
from sklearn.ensemble import ExtraTreesClassifier
trt = ExtraTreesClassifier(n_estimators=1000, 
                           max_features=1, max_depth=3).fit(X_train, y_train)      

# Plot importances
feature_names = df.columns[1:]

importances = pd.DataFrame()
importances["DT"] = pd.Series(grid.best_estimator_.feature_importances_,
                              index=feature_names)
importances["RF"] = pd.Series(rf.feature_importances_, index=feature_names)
importances["TRT"] = pd.Series(trt.feature_importances_, index=feature_names)
importances.plot(kind="barh")

Disclaimer: Importances are measured only through the eyes of the model. _They may not tell the entire nor the same story!_
    

## Partial dependence plots

Relation between the response Y and a subset of features, marginalized over all other features.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble.partial_dependence import plot_partial_dependence

gbrt = GradientBoostingClassifier(n_estimators=100, 
                                  learning_rate=0.1, 
                                  max_leaf_nodes=5)
gbrt.fit(X, y)

plot_partial_dependence(gbrt, X, feature_names=feature_names, features=[0, 6])
plot_partial_dependence(gbrt, X, feature_names=feature_names, features=[(1, 4)])

More detailed information can be found <a href="http://scikit-learn.org/dev/modules/ensemble.html#partial-dependence">here</a>.

## Sample and class weights

- Most learning algorithms assume that input samples are _i.i.d._
- When it is not the case,
    * sample probabilities can be specified through `sample_weight`
    * class probabilities can be specified through `class_weight`
- These are supported by _some_ estimators (but not all)
- Beware that _you are changing the objective function_!

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Make classes equiprobable (shortcut: class_weight="auto"), 
# even if classes are unbalanced
clf = RandomForestClassifier(class_weight={0: 0.5, 1: 0.5}) 

# Specify sample weights
sample_weight = np.random.rand(X_train.shape[0]) 
clf.fit(X_train, y_train, sample_weight=sample_weight)

# Summary

- Scikit-Learn provides all essential tools for machine learning. 
- It is more than training classifiers!
- It integrates within a larger Python scientific ecosystem.
- Try it for yourself!

In [ ]:
questions?

## Further readings


- Scikit-Learn <a href="http://scikit-learn.org/stable/documentation.html">documentation</a>, <a href="http://scikit-learn.org/stable/auto_examples/index.html">example gallery</a>
- PyCon 2015 tutorial: Parts <a href="https://www.youtube.com/watch?v=L7R4HUQ-eQ0">1</a> and <a href="https://www.youtube.com/watch?v=oGqGxvqA9-k">2</a>
- Complementary packages: <a href="http://statsmodels.sourceforge.net/">statsmodel</a>, <a href="http://dan.iel.fm/emcee/current/">emcee</a>, <a href="http://www.nltk.org/">NLTK</a>, <a href="http://deeplearning.net/software/theano/">Theano</a>, <a href="https://github.com/lisa-lab/pylearn2">Pylearn2</a>, <a href="https://github.com/JasperSnoek/spearmint">spearmint</a>, ...